<a href="https://colab.research.google.com/github/Mustafa017/Andela-Bootcamp-/blob/master/cloudboost_scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
from pprint import pprint
import xlsxwriter

In [ ]:
!pip install xlsxwriter

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.5/153.5 kB 3.1 MB/s eta 0:00:00


In [ ]:
url = "https://www.cloudskillsboost.google/course_templates/53?catalog_rank=%7B%22rank%22%3A1%2C%22num_filters%22%3A0%2C%22has_search%22%3Atrue%7D&search_id=25346338"
page = requests.get(url)
page

<Response [200]>

In [ ]:
soup = BeautifulSoup(page.content, "html.parser")
ql_course = soup.find("ql-course")
modules = ql_course.attrs['modules']
print(type(modules))
print("\n\n")
print(modules)

<class 'str'>



[{"id":"59338","title":"Introduction","description":"\u003cp\u003eIn this module, we introduce the course and agenda\u003c/p\u003e","steps":[{"id":"386567","prompt":null,"isOptional":true,"activities":[{"id":"379215","href":null,"isLocked":false,"duration":55000,"title":"Course Introduction","type":"video","isComplete":false,"inProgress":false,"score":null}],"isComplete":false,"allActivitiesRequired":false}],"expanded":false},{"id":"59339","title":"Introduction to Building Batch Data Pipelines","description":"\u003cp\u003eThis module reviews different methods of data loading: EL, ELT and ETL and when to use what\u003c/p\u003e","steps":[{"id":"386568","prompt":null,"isOptional":true,"activities":[{"id":"379216","href":null,"isLocked":false,"duration":69000,"title":"Module introduction","type":"video","isComplete":false,"inProgress":false,"score":null}],"isComplete":false,"allActivitiesRequired":false},{"id":"386569","prompt":null,"isOptional":true,"activities":[{"id":"3

In [ ]:
#convert to json
json_modules = json.loads(modules)
print(type(json_modules))
print("\n\n")
pprint(json_modules[0])

<class 'list'>



{'description': '<p>In this module, we introduce the course and agenda</p>',
 'expanded': False,
 'id': '59338',
 'steps': [{'activities': [{'duration': 55000,
                            'href': None,
                            'id': '379215',
                            'inProgress': False,
                            'isComplete': False,
                            'isLocked': False,
                            'score': None,
                            'title': 'Course Introduction',
                            'type': 'video'}],
            'allActivitiesRequired': False,
            'id': '386567',
            'isComplete': False,
            'isOptional': True,
            'prompt': None}],
 'title': 'Introduction'}


In [ ]:
df = pd.DataFrame(json_modules)
df

,id,title,description,steps,expanded
0,59338,Introduction,"<p>In this module, we introduce the course and...","[{'id': '386567', 'prompt': None, 'isOptional'...",False
1,59339,Introduction to Building Batch Data Pipelines,<p>This module reviews different methods of da...,"[{'id': '386568', 'prompt': None, 'isOptional'...",False
2,59340,Executing Spark on Dataproc,<p>This module shows how to run Hadoop on Data...,"[{'id': '386575', 'prompt': None, 'isOptional'...",False
3,59341,Serverless Data Processing with Dataflow,<p>This module covers using Dataflow to build ...,"[{'id': '386587', 'prompt': None, 'isOptional'...",False
4,59342,Manage Data Pipelines with Cloud Data Fusion a...,<p>This module shows how to manage data pipeli...,"[{'id': '386604', 'prompt': None, 'isOptional'...",False
5,59343,Course Summary,<p>Course Summary</p>,"[{'id': '386620', 'prompt': None, 'isOptional'...",False
6,59344,Course Resources,<p>PDF links to all modules</p>,"[{'id': '386621', 'prompt': None, 'isOptional'...",False


In [ ]:
flatten_df = pd.json_normalize(json_modules, record_path=['steps', ['activities']], meta=['id','title', 'description'], meta_prefix='meta-', record_prefix='record-')
flatten_df.head(3)

,record-id,record-href,record-isLocked,record-duration,record-title,record-type,record-isComplete,record-inProgress,record-score,meta-id,meta-title,meta-description
0,379215,None,False,55000,Course Introduction,video,False,False,None,59338,Introduction,"<p>In this module, we introduce the course and..."
1,379216,None,False,69000,Module introduction,video,False,False,None,59339,Introduction to Building Batch Data Pipelines,<p>This module reviews different methods of da...
2,379217,None,False,220000,"EL, ELT, ETL",video,False,False,None,59339,Introduction to Building Batch Data Pipelines,<p>This module reviews different methods of da...


In [ ]:
flatten_df.drop(['record-href', 'record-isLocked', 'record-isComplete', 'record-inProgress','record-score'], axis=1, inplace=True)

In [ ]:
print(flatten_df.shape)
print("\n\n")
print(flatten_df.info())

(58, 7)



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58 entries, 0 to 57
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   record-id         58 non-null     object
 1   record-duration   58 non-null     int64 
 2   record-title      58 non-null     object
 3   record-type       58 non-null     object
 4   meta-id           58 non-null     object
 5   meta-title        58 non-null     object
 6   meta-description  58 non-null     object
dtypes: int64(1), object(6)
memory usage: 3.3+ KB
None


In [ ]:
grouped_df = flatten_df.groupby(['meta-id','meta-title', 'meta-description'])
print(grouped_df.ngroups)
print("\n\n")
print(grouped_df.size())

7



meta-id  meta-title                                                       meta-description                                                                                                              
59338    Introduction                                                     <p>In this module, we introduce the course and agenda</p>                                                                          1
59339    Introduction to Building Batch Data Pipelines                    <p>This module reviews different methods of data loading: EL, ELT and ETL and when to use what</p>                                 7
59340    Executing Spark on Dataproc                                      <p>This module shows how to run Hadoop on Dataproc, how to leverage Cloud Storage, and how to optimize your Dataproc jobs.</p>    12
59341    Serverless Data Processing with Dataflow                         <p>This module covers using Dataflow to build your data processing pipelines</p>                   

In [ ]:
grouped_df2 = flatten_df.set_index(['meta-id','meta-title', 'meta-description', 'record-id'])
grouped_df2

record-duration  \
meta-id meta-title                                         meta-description                                   record-id                    
59338   Introduction                                       <p>In this module, we introduce the course and ... 379215               55000   
59339   Introduction to Building Batch Data Pipelines      <p>This module reviews different methods of dat... 379216               69000   
                                                                                                              379217              220000   
                                                                                                              379218              168000   
                                                                                                              379219              180000   
                                                                                                              379220              208000   
                                                                                                              379221              428000   
                                                                                                              379222                   0   
59340   Executing Spark on Dataproc                        <p>This module shows how to run Hadoop on Datap... 379223               27000   
                                                                                                              379224              286000   
                                                                                                              379225              602000   
                                                                                                              379226              379000   
                                                                                                              379227              172000   
                                                                                                              379228              567000   
                                                                                                              379229              312000   
                                                                                                              379230              186000   
                                                                                                              379231               28000   
                                                                                                              379232             5400000   
                                                                                                              379233               34000   
                                                                                                              379234                   0   
59341   Serverless Data Processing with Dataflow           <p>This module covers using Dataflow to build y... 379235               53000   
                                                                                                              379236              342000   
                                                                                                              379237              171000   
                                                                                                              379238              228000   
                                                                                                              379239              127000   
                                                                                                              379240              188000   
                                                                                                              379241               10000   
                                                             

In [ ]:
flatten_df.groupby(['meta-id','meta-title', 'meta-description'], dropna=False).groups

{('59338', 'Introduction', '<p>In this module, we introduce the course and agenda</p>'): [0], ('59339', 'Introduction to Building Batch Data Pipelines', '<p>This module reviews different methods of data loading: EL, ELT and ETL and when to use what</p>'): [1, 2, 3, 4, 5, 6, 7], ('59340', 'Executing Spark on Dataproc', '<p>This module shows how to run Hadoop on Dataproc, how to leverage Cloud Storage, and how to optimize your Dataproc jobs.</p>'): [8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19], ('59341', 'Serverless Data Processing with Dataflow', '<p>This module covers using Dataflow to build your data processing pipelines</p>'): [20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39], ('59342', 'Manage Data Pipelines with Cloud Data Fusion and Cloud Composer', '<p>This module shows how to manage data pipelines with Cloud Data Fusion and Cloud Composer.</p>'): [40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55], ('59343', 'Course Summary', '<p>Cour

In [ ]:
with pd.ExcelWriter('demo_GCBS.xlsx', engine='xlsxwriter') as writer:
  grouped_df2.to_excel(writer, sheet_name="CLMG004")

In [ ]:
workbook = writer.book
worksheet = writer.sheets['CLMG004']

In [ ]:
def get_prerequisites():
  pass

In [ ]:
def get_keywords():
  pass

In [ ]:
def misc_info():
  pass